# Assignment 2
## Create a Neural Network using PyTorch for FashionMNIST dataset

In [6]:
# pip install torch torchvision torchaudio

In [ ]:
%matplotlib inline



# NN with PyTorch

PyTorch has two primitives to work with data <https://pytorch.org/docs/stable/data.html>: 
- ``torch.utils.data.DataLoader`` and 
- ``torch.utils.data.Dataset``.

``Dataset`` stores the samples and their corresponding labels, and ``DataLoader`` wraps an iterable around
the ``Dataset``.




### Import the necessary libraries

In [22]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
import matplotlib.pyplot as plt
from torch import nn
import torchvision.models as models

PyTorch offers domain-specific libraries such as 
- `TorchText <https://pytorch.org/text/stable/index.html>`_, 
- `TorchVision <https://pytorch.org/vision/stable/index.html>`_, and 
- `TorchAudio <https://pytorch.org/audio/stable/index.html>`_, 
all of which include datasets. For this tutorial, we  will be using a TorchVision dataset.

The ``torchvision.datasets`` module contains ``Dataset`` objects for many real-world vision data like 
CIFAR, COCO (`full list here <https://pytorch.org/docs/stable/torchvision/datasets.html>`_). In this tutorial, we
use the FashionMNIST dataset. Every TorchVision ``Dataset`` includes two arguments: ``transform`` and
``target_transform`` to modify the samples and labels respectively.



### Download training data from open datasets

In [10]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

### Download test data from open datasets

In [8]:
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

We pass the ``Dataset`` as an argument to ``DataLoader``. This wraps an iterable over our dataset, and supports
automatic batching, sampling, shuffling and multiprocess data loading. Here we define a batch size of 64, i.e. each element 
in the dataloader iterable will return a batch of 64 features and labels.



In [9]:
from torch.utils.data import DataLoader

bs = 64

# Create data loaders

train_dataloader = DataLoader(training_data, batch_size=bs, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=bs, shuffle=True)


--------------




Creating Models
------------------
To define a neural network in PyTorch, we create a class that inherits 
from `nn.Module <https://pytorch.org/docs/stable/generated/torch.nn.Module.html>`_. 

We define the layers of the network in the ``__init__`` function and specify how data will pass through the network in the ``forward`` function. 

To accelerate operations in the neural network, we move it to the GPU if available.



### Get cpu or gpu device for training

In [11]:
if torch.cuda.is_available():
  train_dataloader = tensor.to('cuda')

### Define model
- Flatten the image
- Sequential Layers
    - Linear layer
    - ReLU layer

In [12]:

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

--------------




Optimizing the Model Parameters
----------------------------------------
To train a model, we need
- `loss function` <https://pytorch.org/docs/stable/nn.html#loss-functions>
- `optimizer` <https://pytorch.org/docs/stable/optim.html> 



In [17]:
learning_rate = 5e-3
batch_size = 32
epochs = 25
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and 
backpropagates the prediction error to adjust the model's parameters. 



In [18]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):        
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

We also check the model's performance against the test dataset to ensure it is learning.



In [19]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (*epochs*). During each epoch, the model learns 
parameters to make better predictions. We print the model's accuracy and loss at each epoch; we'd like to see the
accuracy increase and the loss decrease with every epoch.



In [20]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.004433  [    0/60000]
loss: 1.997140  [ 6400/60000]
loss: 1.810298  [12800/60000]
loss: 2.020306  [19200/60000]
loss: 1.807138  [25600/60000]
loss: 1.730054  [32000/60000]
loss: 1.701079  [38400/60000]
loss: 1.654338  [44800/60000]
loss: 1.632774  [51200/60000]
loss: 1.497338  [57600/60000]
Test Error: 
 Accuracy: 48.9%, Avg loss: 0.025584 

Epoch 2
-------------------------------
loss: 1.532953  [    0/60000]
loss: 1.450158  [ 6400/60000]
loss: 1.416183  [12800/60000]
loss: 1.632179  [19200/60000]
loss: 1.625107  [25600/60000]
loss: 1.500213  [32000/60000]
loss: 1.260692  [38400/60000]
loss: 1.172436  [44800/60000]
loss: 1.397828  [51200/60000]
loss: 1.412997  [57600/60000]
Test Error: 
 Accuracy: 51.7%, Avg loss: 0.021745 

Epoch 3
-------------------------------
loss: 1.493867  [    0/60000]
loss: 1.487898  [ 6400/60000]
loss: 1.308868  [12800/60000]
loss: 1.561362  [19200/60000]
loss: 1.297170  [25600/60000]
loss: 1.302579  [32000/600

--------------




In [23]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


Saving Models
-------------
A common way to save a model is to serialize the internal state dictionary (containing the model parameters).



In [24]:
model = models.vgg16(pretrained=True)
torch.save(model.state_dict(), 'model_weights.pth')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

Loading Models
----------------------------

The process for loading a model includes re-creating the model structure and loading
the state dictionary into it. 



In [25]:
model = models.vgg16() # we do not specify pretrained=True, i.e. do not load default weights
model.load_state_dict(torch.load('model_weights.pth'))

<All keys matched successfully>

This model can now be used to make predictions.

